# **신경망 모델 구성하기**

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다. torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다. 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## **학습을 위한 장치 얻기**

가능한 경우 GPU와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. `torch.cuda` 를 사용할 수 있는지 확인하고 그렇지 않으면 CPU를 계속 사용합니다.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


**클래스 정의하기**

신경망 모델을 `nn.Module` 의 하위클래스로 정의하고, `__init__` 에서 신경망 계층들을 초기화합니다. `nn.Module` 을 상속받은 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산들을 구현합니다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork` 의 인스턴스(instance)를 생성하고 이를 `device` 로 이동한 뒤, 구조(structure)를 출력합니다.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들 과 함께 모델의 `forward` 를 실행합니다. `model.forward()` 를 직접 호출하지 마세요!

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10-차원 텐서가 반환됩니다. 원시 예측값을 `nn.Softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


## **모델 계층(Layer)**

FashionMNIST 모델의 계층들을 살펴보겠습니다. 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

### **nn.Flatten**

`nn.Flatten` 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환합니다. (dim=0의 미니배치 차원은 유지됩니다.)

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### **nn.Linear**

선형 계층(nn.Linear)은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈입니다.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### **nn.ReLU**

비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

이 모델에서는 nn.ReLU 를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 5.6669e-01, -6.9860e-02,  1.2043e-01, -7.9264e-02, -7.0249e-02,
          2.2944e-01,  1.9447e-01,  3.5810e-01, -2.6217e-01,  2.8238e-01,
          3.5562e-01, -1.0616e-04,  1.5328e-01, -1.1567e-01,  1.5352e-02,
         -1.5151e-01,  1.1768e-01,  3.4185e-01,  4.8114e-02, -5.3978e-01],
        [ 7.0890e-01, -3.4656e-01,  1.8419e-01, -3.1782e-01, -2.2028e-01,
          2.9129e-01,  2.3312e-01,  4.0550e-01, -3.1934e-02,  2.0046e-02,
          4.1608e-01, -3.2777e-01,  1.3288e-01, -9.4491e-02, -5.7933e-02,
         -1.6608e-01,  1.5072e-01,  6.0528e-01,  6.1251e-01, -3.4601e-01],
        [ 6.9854e-01,  8.7129e-02,  2.2808e-01,  5.7017e-02,  1.0521e-01,
         -3.3207e-02,  5.0425e-01,  3.3203e-01, -1.3538e-02,  4.2857e-01,
          1.8825e-01, -1.1680e-01,  6.0919e-02, -2.2521e-01, -1.8672e-01,
         -2.3130e-01,  4.9835e-01,  4.7148e-01,  9.2942e-01, -4.4996e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.5667, 0.0000, 0.1204, 0.0000, 0.0000,

### **nn.Sequential**

**nn.Sequential** 은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 순차 컨테이너(sequential container)를 사용하여 아래의 `seq_modules` 와 같은 신경망을 빠르게 만들 수 있습니다.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### **nn.Softmax**

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다. logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됩니다. `dim` 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## **모델 매개변수**

신경망 내부의 많은 계층들은 매개변수화(parameterize) 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. `nn.Module` 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 `parameters()` 및 `named_parameters()` 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0002, -0.0316, -0.0235,  ..., -0.0266, -0.0223, -0.0188],
        [-0.0272, -0.0204,  0.0111,  ..., -0.0080,  0.0355, -0.0035]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0285,  0.0239], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0255, -0.0166,  0.0029,  ..., -0.0088, -0.0301,  0.0216],
        [ 0.0294, -0.0349, -0.0176,  ...,  0.0113, -0.0226,  0.0401]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 